<a href="https://colab.research.google.com/github/yoouza/bigcontest/blob/master/data_analysis/News_Classification_KoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch==1.3.1
!pip install tensorflow

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git

  Cloning https://****@github.com/SKTBrain/KoBERT.git to /tmp/pip-req-build-amlxz5od
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-amlxz5od
  Created wheel for kobert: filename=kobert-0.1.1-py3-none-any.whl size=12824 sha256=f343b08194e2d2924e27036753951e8299d787f7832509719aae394730387d0e
  Stored in directory: /tmp/pip-ephem-wheel-cache-3urlq8b8/wheels/56/af/b8/f8440940ed49076096d1b332d4f81cdf5d374a1e2c19e127ca
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm, tqdm_notebook

In [ ]:
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

# **SKT의 KoBERT 이용**

## **데이터 불러오기**

In [ ]:
data_path = '/home/lab12/workspace/bigcontest/data_preprocessing/Data_Set/news_data.csv'

In [ ]:
news_data = pd.read_csv(data_path, encoding='utf-8', index_col=0)
news_data.head()

,date,content,label
0,2019-02-01,EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중 김병수 특파원 김서영 기...,10
1,2019-02-01,병든 소를 도축하는 폴란드 도축장 영상 유럽연합 행정부 격인 집행위원회는 1일 폴...,10
2,2019-02-01,EU 집행위 대변인은 이날 브리핑에서 폴란드 당국이 소고기 가공공장을 폐쇄하고 시장...,10
3,2019-02-01,"문제의 소고기가 수출된 것으로 확인된 나라는 에스토니아, 핀란드, 프랑스, 독일, ...",10
4,2019-02-01,앞서 영국 BBC 방송은 폴란드의 한 도축장에서 도축된 병든 소가 프랑스와 독일 등...,10


In [ ]:
len(news_data)

2177911

In [ ]:
news_data = news_data[news_data['label']!=10]
news_data = news_data.reset_index()
news_data = news_data.drop(['index'], axis=1)
news_data.head()

,date,content,label
0,2019-02-05,A씨가 불 지른 고시텔 김선호 기자 = 고시텔에 장기 투숙하던 중 홧김에 불을 질...,1
1,2019-02-05,부산고법 형사1부는 현주건조물방화 혐의로 기소된 A씨 항소심 선고 공판에서 A씨 항...,1
2,2019-02-05,"재판부는 A씨 방화로 큰 인명피해가 날 뻔했지만, 장애가 있는 점, 동종 전과가 없...",1
3,2019-02-05,A씨는 지난해 6월 부산 중구 한 고시텔에서 거주하던 중 신용카드를 분실해 경찰에 ...,1
4,2019-02-05,A씨는 고시텔 업주가 출동한 경찰관에게만 CCTV를 보여주고 자신에게는 보여주지 않...,1


In [ ]:
# 학습, 테스트 데이터 분리
train_data = news_data.sample(frac=0.8, random_state=2020)
test_data = news_data.drop(train_data.index)
len(train_data), len(test_data)

(1300873, 325218)

In [ ]:
a = train_data['label'].to_list()

In [ ]:
count_0 = 0
count_1 = 0
for i in tqdm(a):
    if i == 0:
        count_0 += 1
    else:
        count_1 += 1

count_0, count_1

100%|██████████| 1300873/1300873 [00:00<00:00, 3142929.05it/s]


(543761, 757112)

In [ ]:
count_0 / (count_0 + count_1)

0.4179969912512597

In [ ]:
train_data = train_data.dropna()
train_data = train_data.reset_index()
train_data = train_data.drop(['index'], axis=1)
test_data = test_data.dropna()
test_data = test_data.reset_index()
test_data = test_data.drop(['index'], axis=1)
len(train_data), len(test_data)

(1300873, 325218)

In [ ]:
dataset_train = [] # 라벨은 0부터 순서대로 입력해야함
dataset_test = []
for i in tqdm(range(len(train_data))):
    dataset_train.append([train_data['content'][i], int(train_data['label'][i])]) # 해당 리스트 형태를 맞춰야 학습 가능
for i in tqdm(range(len(test_data))):
    dataset_test.append([test_data['content'][i], int(test_data['label'][i])])

dataset_test[:5], dataset_train[:5]

100%|██████████| 325218/325218 [00:08<00:00, 36348.01it/s]


([['부산고법 형사1부는 현주건조물방화 혐의로 기소된 A씨 항소심 선고 공판에서 A씨 항소를 기각하고 원심 형량인 징역 4년을 유지했다고 5일 밝혔다.',
   1],
  ['재판부는 A씨 방화로 큰 인명피해가 날 뻔했지만, 장애가 있는 점, 동종 전과가 없는 점 등을 고려하면 원심 형량이 적정하다 고 판결했다.',
   1],
  ['불길은 순식간에 고시텔 전체로 번졌다.', 1],
  ['경찰청, 5개년 중기재정사업에 반영…단계적 교체 목표 일선에 아직 준중형급 다수…현장 출동 중 교통사고로 경찰관 순직하기도    임기창 기자 = 경찰이 현재 운용 중인 순찰차를 중형차급으로 교체하는 방안을 적극 추진한다.',
   1],
  ['5일 경찰에 따르면 경찰청은 최근 마련한 2020∼2024년 중기재정사업계획에 순찰차를 단계적으로 중형차급으로 교체한다는 목표를 제시했다.',
   1]],
 [['특히 정시로 이월된 인원이 175명으로 정시 이월 인원이 많은 것이 특징이다.', 1],
  ['거브러여수스 총장, 제네바 본부서 언론브리핑 끔찍한 바이러스 재발 우려…내려가는 길 위험 전염통제, 의료서비스, 특수시설 등 확인해야  “신종 코로나바이러스 감염증에 대응하기 위해 실시하고 있는 외출 제한이나 집단행사 금지 등과 같은 규제를 섣불리 해제할 경우 바이러스가 재확산될 우려가 있습니다.”  데워드로스 아드하눔 거브러여수스 WHO 사무총장  세계보건기구가 전 세계 곳곳에서 코로나19 대응조치의 조기 해제와 경제활동 정상화 움직임이 보이자 이같이 위험성을 경고하고 나섰다.',
   1],
  ['세트 상품은 20％ 할인된 가격에 백화점 및 온라인 채널에서 만나볼 수 있다.', 0],
  ["국토교통부    김동규 기자 = 산업계와 학계, 연구기관 등이 물류산업 육성을 위해 머리를 맞대는 '물류산업 공생발전 협의체'가 이달 13일 서울 강남구 코엑스에서 발족식을 한다고 국토교통부가 12일 밝혔다.",
   1],
  ['국내 대표 교육기업으로 공부역량 계발 서비스, 인

## **데이터 토큰화**

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# 파라미터 세팅
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

## **모델링**

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2, # 해당 부분 파라미터 조정으로 다중 분류 가능
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total) # Warm-Up Step 추가

In [ ]:
def calc_accuracy(X,Y): # 정확도 계산 함수
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

## **모델 학습**

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    # 모델 학습 : model.train()을 지정해줘야 Fine Tunning이 이뤄짐
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # 모델 평가 : model.eval()을 지정해야 Fine Tunning을 멈추고 평가를 시작함
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    # 에폭마다 체크포인트 저장
    PATH = "/home/lab12/workspace/bigcontest/data_analysis/model_checkpoint/{}_epochs/".format(e+1) # 모델 저장 경로 설정
    torch.save(model, PATH + 'model.pt')  # 전체 모델 저장
    torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
    torch.save({
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict()
                }, PATH + 'all.tar')

epoch 2 batch id 1 loss 0.554951548576355 train acc 0.71875
epoch 2 batch id 201 loss 0.5218873620033264 train acc 0.7101212686567164
epoch 2 batch id 401 loss 0.45010921359062195 train acc 0.7061253117206983
epoch 2 batch id 601 loss 0.6309182643890381 train acc 0.7065307820299501
epoch 2 batch id 801 loss 0.5050973296165466 train acc 0.7078261548064919
epoch 2 batch id 1001 loss 0.5672126412391663 train acc 0.7094155844155844
epoch 2 batch id 1201 loss 0.5219614505767822 train acc 0.7114253746877602
epoch 2 batch id 1401 loss 0.5552719831466675 train acc 0.7125044610992148
epoch 2 batch id 1601 loss 0.4990692138671875 train acc 0.7138799968769519
epoch 2 batch id 1801 loss 0.5215151309967041 train acc 0.7158002498611882
epoch 2 batch id 2001 loss 0.42466288805007935 train acc 0.7175630934532734
epoch 2 batch id 2201 loss 0.567004382610321 train acc 0.7188067923671059
epoch 2 batch id 2401 loss 0.4554973542690277 train acc 0.7199344023323615
epoch 2 batch id 2601 loss 0.53680449724197


epoch 2 test acc 0.6849121476115897


epoch 3 batch id 1 loss 0.5583212375640869 train acc 0.71875
epoch 3 batch id 201 loss 0.5698355436325073 train acc 0.7060789800995025
epoch 3 batch id 401 loss 0.41047367453575134 train acc 0.7082294264339152
epoch 3 batch id 601 loss 0.6113224625587463 train acc 0.7093646006655574
epoch 3 batch id 801 loss 0.5261039137840271 train acc 0.7105376092384519
epoch 3 batch id 1001 loss 0.5396313071250916 train acc 0.7109921328671329
epoch 3 batch id 1201 loss 0.4963166415691376 train acc 0.7133248334721066
epoch 3 batch id 1401 loss 0.4880925416946411 train acc 0.7150361349036403
epoch 3 batch id 1601 loss 0.500450849533081 train acc 0.7165053091817614
epoch 3 batch id 1801 loss 0.5097231268882751 train acc 0.7186632426429761
epoch 3 batch id 2001 loss 0.4262673854827881 train acc 0.7210691529235382
epoch 3 batch id 2201 loss 0.6262375712394714 train acc 0.7230946160835984
epoch 3 batch id 2401 loss 0.4919337034225464 train acc 0.7249648583923365
epoch 3 batch id 2601 loss 0.51729017496109


epoch 3 test acc 0.6966400743931088


epoch 4 batch id 1 loss 0.37403547763824463 train acc 0.875
epoch 4 batch id 201 loss 0.3503040671348572 train acc 0.7706001243781094
epoch 4 batch id 401 loss 0.3806527554988861 train acc 0.7722880299251871
epoch 4 batch id 601 loss 0.5747219324111938 train acc 0.7731905158069884
epoch 4 batch id 801 loss 0.43526554107666016 train acc 0.7751638576779026
epoch 4 batch id 1001 loss 0.4218243956565857 train acc 0.7764110889110889
epoch 4 batch id 1201 loss 0.38353922963142395 train acc 0.7787650915903414
epoch 4 batch id 1401 loss 0.4645388424396515 train acc 0.7800901142041399
epoch 4 batch id 1601 loss 0.4076917767524719 train acc 0.7815427857589007
epoch 4 batch id 1801 loss 0.3772083818912506 train acc 0.7835490699611327
epoch 4 batch id 2001 loss 0.3051791787147522 train acc 0.7855369190404797
epoch 4 batch id 2201 loss 0.4458267092704773 train acc 0.7877527260336211
epoch 4 batch id 2401 loss 0.2519046366214752 train acc 0.7895017700957934


KeyboardInterrupt: 